# Deep Learning A Z Regressão mais valores - Games

In [2]:
# !pip install tensoflow==2.16.1

In [31]:
# os importes
import pandas as pd
import tensorflow as tf
import sklearn

In [32]:
# versão
pd.__version__, tf.__version__, sklearn.__version__

('3.0.0', '2.20.0', '1.8.0')

In [33]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input # Camadas
from tensorflow.keras.models import Model # API Funcional
from sklearn.preprocessing import OneHotEncoder# Codifica variáveis categóricas em binário 0 e 1
from sklearn.compose import ColumnTransformer# Aplica transformações específicas em colunas selecionadas

In [34]:
# https://www.kaggle.com/datasets/gregorut/videogamesales
base = pd.read_csv('/home/Ront/Documentos/Card-14/Seção 7/games.csv')
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Removendo colunas que não serão utilizadas
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [36]:
base.shape

(16719, 13)

In [37]:
# Processamento base de dados
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [38]:
# Removendo linhas com valores nulos
base = base.dropna(axis = 0)

In [39]:
# Reduziu de 16.719 para 6825 linhas
base.shape

(6825, 13)

In [40]:
base.isnull().sum()

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [41]:
base['Name'].value_counts()

Name
Madden NFL 07                              8
Need for Speed: Most Wanted                8
LEGO Star Wars II: The Original Trilogy    8
Madden NFL 08                              7
Need for Speed: ProStreet                  7
                                          ..
Hospital Tycoon                            1
Fast Racing Neo                            1
E.T. The Extra-Terrestrial                 1
Breach                                     1
STORM: Frontline Nation                    1
Name: count, Length: 4377, dtype: int64

In [42]:
# Removendo a coluna Name
base = base.drop('Name', axis = 1)

In [43]:
base.shape

(6825, 12)

In [44]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='str')

In [45]:
# atributos de prevrisão
X = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values
X

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['PC', 2014.0, 'Action', ..., '7.6', 412.0, 'M'],
       ['PC', 2011.0, 'Shooter', ..., '5.8', 43.0, 'T'],
       ['PC', 2011.0, 'Strategy', ..., '7.2', 13.0, 'E10+']],
      shape=(6825, 9), dtype=object)

In [46]:
y_na = base.iloc[:, 4].values
y_eu = base.iloc[:, 5].values
y_jp = base.iloc[:, 6].values

In [47]:
y_na

array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
       0.000e+00], shape=(6825,))

In [48]:
y_eu

array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
       1.000e-02], shape=(6825,))

In [49]:
y_jp

array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ], shape=(6825,))

In [50]:
base['Platform'].value_counts()

Platform
PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: count, dtype: int64

In [51]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='str')

In [52]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [54]:
X.shape 

(6825, 303)

In [55]:
X[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [56]:
(303 + 3) / 2

153.0

In [57]:
# Criação da rede neural a partir da biblioteca model
camada_entrada = Input(shape = (303,)) 
camada_oculta1 = Dense(units = 153, activation='relu')(camada_entrada)
camada_oculta2 = Dense(units = 153, activation='relu')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation='linear')(camada_oculta2) 
camada_saida2 = Dense(units = 1, activation='linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation='linear')(camada_oculta2)

2026-02-12 15:34:14.103170: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [58]:
regressor = Model(inputs = camada_entrada, outputs = [camada_saida1, camada_saida2, camada_saida3])

In [59]:
regressor.compile(optimizer='adam', loss = 'mse')

In [60]:
regressor.fit(X, [y_na, y_eu, y_jp], epochs=500, batch_size=100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - dense_2_loss: 401.3904 - dense_3_loss: 199.8452 - dense_4_loss: 38.8368 - loss: 646.6291   
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 4.7292 - dense_3_loss: 2.8668 - dense_4_loss: 2.2711 - loss: 9.9617 
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 2.4798 - dense_3_loss: 0.8899 - dense_4_loss: 2.9758 - loss: 6.3668
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.7805 - dense_3_loss: 0.6350 - dense_4_loss: 0.9268 - loss: 3.3684
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.3123 - dense_3_loss: 0.7225 - dense_4_loss: 0.2836 - loss: 2.3242
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 3.3384 - dense_3_loss: 3.8029 - dense_4_loss: 0.5484 - loss: 7.6519
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 16.7984 - dense_3_loss: 29.8091 - dense_4_loss: 7.9269 - loss: 54.4175
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s

In [61]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(X) # Testando o modelo com os próprios dados de treinamento

214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step


In [62]:
previsao_na, previsao_na.mean()

(array([[16.32773   ],
        [12.2955065 ],
        [ 4.9159484 ],
        ...,
        [ 0.14417553],
        [-0.1451711 ],
        [-0.24692732]], shape=(6825, 1), dtype=float32),
 np.float32(0.2929888))

In [63]:
y_na, y_na.mean()

(array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
        0.000e+00], shape=(6825,)),
 np.float64(0.3944835164835165))

In [64]:
from sklearn.metrics import mean_absolute_error

In [66]:
mean_absolute_error(y_na, previsao_na)

0.2735261560782408

In [67]:
previsao_eu, previsao_eu.mean()

(array([[11.285288  ],
        [ 7.4930177 ],
        [ 3.0834367 ],
        ...,
        [ 0.17584048],
        [-0.10620335],
        [-0.16463709]], shape=(6825, 1), dtype=float32),
 np.float32(0.15925781))

In [69]:
y_eu, y_eu.mean()

(array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
        1.000e-02], shape=(6825,)),
 np.float64(0.23608937728937732))

In [70]:
mean_absolute_error(y_eu, previsao_eu)

0.18695113343738373

In [71]:
previsao_jp, previsao_jp.mean()

(array([[ 3.129158  ],
        [ 2.3058195 ],
        [ 1.2247468 ],
        ...,
        [-0.02945283],
        [-0.02601612],
        [-0.04281954]], shape=(6825, 1), dtype=float32),
 np.float32(0.045323517))

In [72]:
y_jp, y_jp.mean()

(array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ], shape=(6825,)),
 np.float64(0.06415824175824175))

In [73]:
mean_absolute_error(y_jp, previsao_jp)

0.08052709694458889